In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sympy as sp
from math import factorial
from tqdm import tqdm
import csv
from scipy.sparse import csr_matrix
from scipy.sparse import identity
from scipy.sparse.linalg import inv

In [4]:
A = 4
N = 5
S = int(factorial(A+2+A**2+N)/factorial(A+2+A**2)/factorial(N))
S

80730

In [4]:
# generate state dictionary direrctly
# rather than list first than dictionary
# A = 2
# N = 2
state = {}
index = [0]

def temp(x, i, index):
    x.append(i)
    generate_R(x.copy(), index)
    
def generate_R(s, index):
    if len(s)==A+3+A**2 and sum(s)==N:
        state[tuple(s)] = index[0]
        index[0] += 1
    elif len(s)>A+3+A**2 or sum(s)>N:
        return 0
    else:
        for i in range(N+1):
            temp(s.copy(), i, index)
            

for i in range(N+1):
    generate_R([i], index)
n_state = len(state)
n_queue = A+3+A**2
n_queue, n_state

KeyboardInterrupt: 

In [133]:
i = 0
for s in state:
    print(s, state[s])
    i+=1
    if i == 9: break

(0, 0, 0, 0, 0, 0, 0, 0, 8) 0
(0, 0, 0, 0, 0, 0, 0, 1, 7) 1
(0, 0, 0, 0, 0, 0, 0, 2, 6) 2
(0, 0, 0, 0, 0, 0, 0, 3, 5) 3
(0, 0, 0, 0, 0, 0, 0, 4, 4) 4
(0, 0, 0, 0, 0, 0, 0, 5, 3) 5
(0, 0, 0, 0, 0, 0, 0, 6, 2) 6
(0, 0, 0, 0, 0, 0, 0, 7, 1) 7
(0, 0, 0, 0, 0, 0, 0, 8, 0) 8


In [134]:
#pij
Pij = [[0.3, 0.7],
       [0.7, 0.3]]
Broken = 0.15

#rate
#central
# ['lambda0','lambda1','car_move','gamma','phi','delta','r00','r01','r10','r11','broken','period']
Rate = [4.0, 5.0, 1.0, 2.0, 1.0, 1.0, 1.0, 1.0, 1.0]

In [135]:
# 使用scipy.sparse.csr_sparse
# generate R matrix
# 换个视角，先确定哪些队列不为0，只有这些不为0的队列会使系统的状态发生改变

#R = csr_matrix((S,S), dtype=np.float)
Row, Col, Data = [], [], []
for i, s in enumerate(state):
    n, col, data = 0, [], []
    v = 0
    s = list(s)
    for j, q in enumerate(s):
        #根据瞬时转移率方程组确定r的行向量
        if q != 0:
            #更新v
            if j<A+3: v += Rate[j]
            else: v += q*Rate[j]
            #更新qij
            if j < A:      #骑行
                for t in range(2):
                    ts = s.copy()
                    ts[j] -= 1
                    ts[t+j*2+A+3] += 1
                    c = state[tuple(ts)]
                    if c in col:
                        data[col.index(c)] += Rate[j]*Pij[j][t]
                    else:
                        n+=1
                        col.append(c)
                        data.append(Rate[j]*Pij[j][t])
                
            elif j in [A,A+1]:     #收集
                ts = s.copy()
                ts[j] -= 1
                ts[j+1] += 1
                c = state[tuple(ts)]
                if c in col:
                    data[col.index(c)] += Rate[j]
                else:
                    n+=1
                    col.append(c)
                    data.append(Rate[j])

            elif j == A+2: #维修
                for t in range(A):
                    ts = s.copy()
                    ts[j] -= 1
                    ts[t] += 1
                    c = state[tuple(ts)]
                    if c in col:
                        data[col.index(c)] += Rate[j]*(1/A)
                    else:
                        n+=1
                        col.append(c)
                        data.append(Rate[j]*(1/A))

            else:          #骑完
                for t in [(j-1)%2, A]:
                    ts = s.copy()
                    ts[j] -= 1
                    ts[t] += 1
                    
                    c = state[tuple(ts)]
                    if c in col:
                        if t == A: data[col.index(c)] += q*Rate[j]*Broken
                        else: data[col.index(c)] += q*Rate[j]*(1-Broken)
                    else:
                        n+=1
                        col.append(c)
                        if t == A: data.append(q*Rate[j]*Broken)
                        else: data.append(q*Rate[j]*(1-Broken))
                
        else:
            continue
    n+=1
    col.append(i)
    data.append(-v)
    Row+=[i]*n
    Col+=col
    Data+=data

R = csr_matrix((Data, (Row, Col)), dtype=np.float) #.toarray()
#R[:10]

In [136]:
R.toarray()[-10:]

array([[ 0.  ,  0.  ,  0.  , ...,  0.  ,  0.  ,  0.  ],
       [ 0.  ,  0.  ,  0.  , ...,  0.15,  0.85,  0.  ],
       [ 0.  ,  0.  ,  0.  , ...,  0.15,  0.  ,  0.85],
       ...,
       [ 0.  ,  0.  ,  0.  , ..., -5.  ,  0.  ,  0.  ],
       [ 0.  ,  0.  ,  0.  , ...,  0.  , -9.  ,  0.  ],
       [ 0.  ,  0.  ,  0.  , ...,  0.  ,  0.  , -4.  ]])

In [20]:
with open('Rate.csv', 'w') as fo:
    writer = csv.writer(fo)
    for row in R:
        writer.writerow(row)


$P(t) = e^{\boldsymbol{R}t} \approx \left (I - \boldsymbol{R}\frac{t}{n} \right)^{-n}$

In [ ]:
t, n = 2000000, 10
#mr = np.mat(R)
P = (inv(identity(S) - R*(t/n)))**n

/Users/valarian/python/miniconda3/lib/python3.7/site-packages/scipy/sparse/linalg/dsolve/linsolve.py:296: SparseEfficiencyWarning: splu requires CSC matrix format
  warn('splu requires CSC matrix format', SparseEfficiencyWarning)
/Users/valarian/python/miniconda3/lib/python3.7/site-packages/scipy/sparse/linalg/dsolve/linsolve.py:203: SparseEfficiencyWarning: spsolve is more efficient when sparse b is in the CSC matrix format
  'is in the CSC matrix format', SparseEfficiencyWarning)


In [118]:
p_r1 = list(P.toarray()[0])

In [123]:
#比较没用稀疏矩阵和稀疏矩阵的结果
df = pd.read_csv('/Users/valarian/SJTU/SJTU/毕业论文/Data/code/nosparse_2_2Pij.csv', index_col=0)
for i in range(n_state):
    print(int(df.loc[0][i]*1000) == int(p_r1[i]*1000))
    #print(df.loc[0][i], p_r1[i])

True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True


In [118]:
P_ = np.array(P)
sum(P_[0][:])

0.9999999997773532

In [115]:
pd.DataFrame(P).to_csv('Pij.csv')

In [117]:
sum(stat.seg[:36])

0.8473056719635333

In [105]:
df.t.max()

199999.9070970641

In [ ]:
from scipy.sparse import csr_matrix
a = csr_matrix(([1,2,3], ([0,1,1],[1,0,1])), dtype=np.float)

from scipy.sparse.linalg import inv
%timeit (inv(a))**10

b = inv(a)
def test(b):
    for i in range(10):
        b = b.multiply(b)
%timeit test(b)

In [108]:
state

[[0, 0, 0, 0, 0, 0, 0, 0, 2],
 [0, 0, 0, 0, 0, 0, 0, 1, 1],
 [0, 0, 0, 0, 0, 0, 0, 2, 0],
 [0, 0, 0, 0, 0, 0, 1, 0, 1],
 [0, 0, 0, 0, 0, 0, 1, 1, 0],
 [0, 0, 0, 0, 0, 0, 2, 0, 0],
 [0, 0, 0, 0, 0, 1, 0, 0, 1],
 [0, 0, 0, 0, 0, 1, 0, 1, 0],
 [0, 0, 0, 0, 0, 1, 1, 0, 0],
 [0, 0, 0, 0, 0, 2, 0, 0, 0],
 [0, 0, 0, 0, 1, 0, 0, 0, 1],
 [0, 0, 0, 0, 1, 0, 0, 1, 0],
 [0, 0, 0, 0, 1, 0, 1, 0, 0],
 [0, 0, 0, 0, 1, 1, 0, 0, 0],
 [0, 0, 0, 0, 2, 0, 0, 0, 0],
 [0, 0, 0, 1, 0, 0, 0, 0, 1],
 [0, 0, 0, 1, 0, 0, 0, 1, 0],
 [0, 0, 0, 1, 0, 0, 1, 0, 0],
 [0, 0, 0, 1, 0, 1, 0, 0, 0],
 [0, 0, 0, 1, 1, 0, 0, 0, 0],
 [0, 0, 0, 2, 0, 0, 0, 0, 0],
 [0, 0, 1, 0, 0, 0, 0, 0, 1],
 [0, 0, 1, 0, 0, 0, 0, 1, 0],
 [0, 0, 1, 0, 0, 0, 1, 0, 0],
 [0, 0, 1, 0, 0, 1, 0, 0, 0],
 [0, 0, 1, 0, 1, 0, 0, 0, 0],
 [0, 0, 1, 1, 0, 0, 0, 0, 0],
 [0, 0, 2, 0, 0, 0, 0, 0, 0],
 [0, 1, 0, 0, 0, 0, 0, 0, 1],
 [0, 1, 0, 0, 0, 0, 0, 1, 0],
 [0, 1, 0, 0, 0, 0, 1, 0, 0],
 [0, 1, 0, 0, 0, 1, 0, 0, 0],
 [0, 1, 0, 0, 1, 0, 0, 0, 0],
 [0, 1, 0,

In [104]:
df = pd.read_csv('/Users/valarian/SJTU/SJTU/毕业论文/Data/simulationResult/central/data/central_QN_Pstate.csv',
                names=['state', 't'], index_col=None, skiprows=2)
df['seg'] = df.t.diff().shift(-1)
df = df[:-1]
stat = df.drop(labels='t', axis=1)
#stat.tail()
stat = stat.groupby(stat.state).sum()
stat.seg/=df.t.max()
stat

,seg
state,
"(0, 0, 0, 0, 0, 0, 0, 0, 2)",0.008242
"(0, 0, 0, 0, 0, 0, 0, 1, 1)",0.039566
"(0, 0, 0, 0, 0, 0, 0, 2, 0)",0.046327
"(0, 0, 0, 0, 0, 0, 1, 0, 1)",0.039542
"(0, 0, 0, 0, 0, 0, 1, 1, 0)",0.093161
"(0, 0, 0, 0, 0, 0, 2, 0, 0)",0.046454
"(0, 0, 0, 0, 0, 1, 0, 0, 1)",0.017016
"(0, 0, 0, 0, 0, 1, 0, 1, 0)",0.040178
"(0, 0, 0, 0, 0, 1, 1, 0, 0)",0.039627


In [48]:
# generate R matrix
# 这个版本里尝试逐个状态分量求转移率
R = []
for i, s in enumerate(state):
    r = [0]*n_state
    r[i] = V[i]
    for j, q in enumerate(s):
        #根据瞬时转移率方程组确定r的行向量
        if j < A:      #骑行
            if q == 0: #只用考虑增加的情况
                for l in [A+2, A+3+j, A+5+j]:
                    ts = s.copy()
                    ts[j] = 1
                    if l == A+2:
                        ts[A+2] -= 1
                        r[state_index[tuple(ts)]] = V[state_index[tuple(ts)]]*0.5
                    elif l == A+3+j:
                        ts[l] -= 1
                        r[state_index[tuple(ts)]] = V[state_index[tuple(ts)]]*pij[][j]
                    else:
                        ts[l] -= 1
                        r[state_index[tuple(ts)]] = V[state_index[tuple(ts)]]*pij[][j]
            elif q<N:  #增加、减少
                pass
            else:      #只用考虑减少的情况
                pass
            
        elif j==A:     #收集
            if q == 0: #只用考虑增加的情况
                pass
            elif q<N:  #增加、减少
                pass
            else:      #只用考虑减少的情况
                pass
            
        elif j == A+1: #维修
            if q == 0: #只用考虑增加的情况
                pass
            elif q<N:  #增加、减少
                pass
            else:      #只用考虑减少的情况
                pass
            
        elif j == A+2: #分布
            if q == 0: #只用考虑增加的情况
                pass
            elif q<N:  #增加、减少
                pass
            else:      #只用考虑减少的情况
                pass
            
        else:          #骑完
            elif j == A+1: #维修
            if q == 0: #只用考虑增加的情况
                pass
            elif q<N:  #增加、减少
                pass
            else:      #只用考虑减少的情况
                pass
            
    R.append(r)
    
r[:10]

SyntaxError: unexpected EOF while parsing (<ipython-input-48-b7957739c88f>, line 6)

In [ ]:
# 看看不同的A，N状态空间的变化

def temp(A, N, x, i):
    x.append(i)
    generate_R(A, N, x.copy())
    
def generate_R(A, N, s):
    if len(s)==A+3+A**2 and sum(s)==N:
        #print('add')
        state.append(s)
    elif len(s)>A+3+A**2 or sum(s)>N:
        return 0
    else:
        for i in range(N+1):
            temp(A, N, s.copy(), i)

result = []
for a in tqdm(range(2,10)):
    for n in range(1,15):
        state = []
        A, N = a, n
        for i in range(N+1):
            generate_R(A, N, [i])
        n_state = len(state)
        n_queue = A+3+A**2
        result.append([n_queue, n_state])

In [72]:
# construct a dictionary for state
state_index = {}
for i, s in enumerate(state):
    state_index[tuple(s)] = i

# for i in state[:10]:
#     print(state_index[tuple(i)])

In [7]:
#实际上不需要计算V，在计算R矩阵的过程中V的值可以同时被计算
# generate vi
V = []
for s in state:
    rate = 0
    for n,_ in enumerate(s):
        if n<A+3: 
            if _>0: rate += Rate[n]
        else: rate += _*Rate[n]
    V.append(rate)

state[:10], V[:10]

([[0, 0, 0, 0, 0, 0, 0, 0, 2],
  [0, 0, 0, 0, 0, 0, 0, 1, 1],
  [0, 0, 0, 0, 0, 0, 0, 2, 0],
  [0, 0, 0, 0, 0, 0, 1, 0, 1],
  [0, 0, 0, 0, 0, 0, 1, 1, 0],
  [0, 0, 0, 0, 0, 0, 2, 0, 0],
  [0, 0, 0, 0, 0, 1, 0, 0, 1],
  [0, 0, 0, 0, 0, 1, 0, 1, 0],
  [0, 0, 0, 0, 0, 1, 1, 0, 0],
  [0, 0, 0, 0, 0, 2, 0, 0, 0]],
 [2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0])

In [55]:
# generate R matrix
# 换个视角，先确定哪些队列不为0，只有这些不为0的队列会使系统的状态发生改变

R1 = []
for i, s in enumerate(state):
    r = [0]*n_state
    v = 0
    #print(s)
    for j, q in enumerate(s):
        #根据瞬时转移率方程组确定r的行向量
        if q != 0:
            #更新v
            if j<A+3: v += Rate[j]
            else: v += q*Rate[j]
            #更新qij
            if j < A:      #骑行
                for t in range(2):
                    ts = s.copy()
                    ts[j] -= 1
                    ts[t+j*2+A+3] += 1
                    r[state_index[tuple(ts)]] += Rate[j]*Pij[j][t]
                
            elif j in [A,A+1]:     #收集
                ts = s.copy()
                ts[j] -= 1
                ts[j+1] += 1
                r[state_index[tuple(ts)]] += Rate[j]

            elif j == A+2: #维修
                for t in range(A):
                    ts = s.copy()
                    ts[j] -= 1
                    ts[t] += 1
                    r[state_index[tuple(ts)]] += Rate[j]*(1/A)

            else:          #骑完
                for t in [(j-1)%2, A]:
                    ts = s.copy()
                    #print(ts)
                    ts[j] -= 1
                    ts[t] += 1
                    #print(ts)
                    if t == A: r[state_index[tuple(ts)]] += q*Rate[j]*Broken
                    else: r[state_index[tuple(ts)]] += q*Rate[j]*(1-Broken)
                
        else:
            continue
            
    r[i] = -v
    R1.append(r)
    
#R[:10]